In [ ]:
import pandas as pd
import ollama
import os
import re

# 경로 설정
input_dir = "./Data/None"
output_dir = "./Data/ZeroShot"
os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리 생성

# 클래스 이름 한국어로 매핑
hate_type_map = {
    "none": "없음",
    "offensive": "모욕",
    "hate": "혐오"
}

# 데이터 생성 함수
def generate_data(hate_type, additional_count, existing_data, output_path):
    generated_set = set(existing_data['comments'].dropna())  # 중복 방지용 집합
    hate_type_kor = hate_type_map.get(hate_type, hate_type)  # 한국어로 변환

    while additional_count > 0:
        try:
            # Train 데이터에서 해당 클래스의 랜덤 예시 3개 추출
            examples = existing_data[existing_data['hate'] == hate_type]['comments'].dropna().sample(3).tolist()
            prompt = f"""
            아래의 형식에 맞추어 {hate_type_kor} 혐오유형의 새로운 문장을 하나 작성해줘.
            **출력 형식: {{새로운 문장}}**
            다른 형식의 출력은 절대 사용하지마.

            이제 새로운 문장을 작성해줘.
            """

            # Ollama 호출
            response = ollama.chat(model="exaone3.5:2.4b", messages=[
                {
                    "role": "user",
                    "content": prompt.strip(),
                }
            ])
            generated_text = response['message']['content'].strip()

            # 정규 표현식으로 { } 안의 내용 추출
            match = re.search(r"\{(.+?)\}", generated_text)
            if match:
                new_comment = match.group(1).strip()

                # 중복 확인
                if new_comment in generated_set:
                    continue

                # 새로운 데이터 추가
                new_row = {"comments": new_comment, "hate": hate_type}
                existing_data = pd.concat([existing_data, pd.DataFrame([new_row])], ignore_index=True)
                generated_set.add(new_comment)
                additional_count -= 1

                # 중간 저장
                existing_data.to_csv(output_path, index=False, encoding="utf-8-sig")

        except Exception as e:
            print(f"오류 발생: {e}")
            continue

    return existing_data

# 각 파일 처리
for file_name in sorted(os.listdir(input_dir)):
    if file_name.endswith(".csv"):
        input_path = os.path.join(input_dir, file_name)
        output_file = f"ZeroShot_{file_name.split('_')[-1]}"
        output_path = os.path.join(output_dir, output_file)

        # 파일 로드
        gr_data = pd.read_csv(input_path)

        # 클래스별 100% 추가 생성
        current_counts = gr_data['hate'].value_counts()
        for hate_type, count in current_counts.items():
            additional_count = count  # 100% 증가
            gr_data = generate_data(hate_type, additional_count, gr_data, output_path)

        # 클래스 추가 후 확인
        final_counts = gr_data['hate'].value_counts()
        print(f"{output_path} - 모든 클래스의 데이터가 100% 증가했습니다. 최종 분포: {final_counts}")

# 최종 완료 메시지
print("모든 파일에 대한 데이터 생성 및 저장이 완료되었습니다.")


In [ ]:
import pandas as pd
import ollama
import os
import re

# 경로 설정
input_dir = "./Data/None"
output_dir = "./Data/OneShot"
os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리 생성

# 클래스 이름 한국어로 매핑
hate_type_map = {
    "none": "없음",
    "offensive": "모욕",
    "hate": "혐오"
}

# 데이터 생성 함수
def generate_data(hate_type, additional_count, existing_data, output_path):
    generated_set = set(existing_data['comments'].dropna())  # 중복 방지용 집합
    hate_type_kor = hate_type_map.get(hate_type, hate_type)  # 한국어로 변환

    # 해당 hate 유형의 랜덤 예시 하나 추출
    example = existing_data[existing_data['hate'] == hate_type]['comments'].dropna().sample(1).iloc[0]

    while additional_count > 0:
        try:
            # 프롬프트 구성
            prompt = f"""
            아래는 {hate_type_kor} 유형의 문장 예시야:
            예시: "{example}"

            위 예시를 참고하여 {hate_type_kor} 유형의 새로운 문장을 하나 작성해줘.
            **출력 형식: {{새로운 문장}}**
            다른 형식의 출력은 절대 사용하지마.

            이제 새로운 문장을 작성해줘.
            """

            # Ollama 호출
            response = ollama.chat(model="exaone3.5:2.4b", messages=[
                {
                    "role": "user",
                    "content": prompt.strip(),
                }
            ])
            generated_text = response['message']['content'].strip()

            # 정규 표현식으로 { } 안의 내용 추출
            match = re.search(r"\{(.+?)\}", generated_text)
            if match:
                new_comment = match.group(1).strip()

                # 중복 확인
                if new_comment in generated_set:
                    continue

                # 새로운 데이터 추가
                new_row = {"comments": new_comment, "hate": hate_type}
                existing_data = pd.concat([existing_data, pd.DataFrame([new_row])], ignore_index=True)
                generated_set.add(new_comment)
                additional_count -= 1

                # 중간 저장
                existing_data.to_csv(output_path, index=False, encoding="utf-8-sig")

        except Exception as e:
            print(f"오류 발생: {e}")
            continue

    return existing_data

# 각 파일 처리
for file_name in sorted(os.listdir(input_dir)):
    if file_name.endswith(".csv"):
        input_path = os.path.join(input_dir, file_name)
        output_file = f"OneShot_{file_name.split('_')[-1]}"
        output_path = os.path.join(output_dir, output_file)

        # 파일 로드
        gr_data = pd.read_csv(input_path)

        # 클래스별 100% 추가 생성
        current_counts = gr_data['hate'].value_counts()
        for hate_type, count in current_counts.items():
            additional_count = count  # 100% 증가
            gr_data = generate_data(hate_type, additional_count, gr_data, output_path)

        # 클래스 추가 후 확인
        final_counts = gr_data['hate'].value_counts()
        print(f"{output_path} - 모든 클래스의 데이터가 100% 증가했습니다. 최종 분포: {final_counts}")

# 최종 완료 메시지
print("모든 파일에 대한 데이터 생성 및 저장이 완료되었습니다.")

In [38]:
import pandas as pd
import ollama
import os
import re

# 경로 설정
input_dir = "./Data/None"
output_dir = "./Data/ThreeShot"
os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리 생성

# 클래스 이름 한국어로 매핑
hate_type_map = {
    "none": "없음",
    "offensive": "모욕",
    "hate": "혐오"
}

# 데이터 생성 함수
def generate_data(hate_type, additional_count, existing_data, output_path):
    generated_set = set(existing_data['comments'].dropna())  # 중복 방지용 집합
    hate_type_kor = hate_type_map.get(hate_type, hate_type)  # 한국어로 변환

    # 해당 hate 유형의 예시 순서대로 추출
    examples = existing_data[existing_data['hate'] == hate_type]['comments'].dropna().tolist()
    example_count = len(examples)

    while additional_count > 0:
        try:
            # 순서대로 3개의 예시를 구성
            start_index = (example_count - additional_count) % example_count
            current_examples = examples[start_index:start_index + 3]

            # 순환 처리로 3개가 안되면 앞에서 추가
            while len(current_examples) < 3:
                current_examples.append(examples[len(current_examples) % example_count])

            # 프롬프트 구성
            prompt = f"""
            아래는 {hate_type_kor} 유형의 문장 예시야:
            예시 1: "{current_examples[0]}"
            예시 2: "{current_examples[1]}"
            예시 3: "{current_examples[2]}"

            위 예시를 참고하여 {hate_type_kor} 유형의 새로운 문장을 하나 작성해줘.
            **출력 형식: {{새로운 문장}}**
            다른 형식의 출력은 포함하지 말아줘.

            이제 새로운 문장을 작성해줘.
            """

            # Ollama 호출
            response = ollama.chat(model="exaone3.5:2.4b", messages=[
                {
                    "role": "user",
                    "content": prompt.strip(),
                }
            ])
            generated_text = response['message']['content'].strip()

            # 정규 표현식으로 { } 안의 내용 추출
            match = re.search(r"\{(.+?)\}", generated_text)
            if match:
                new_comment = match.group(1).strip()

                # 중복 확인
                if new_comment in generated_set:
                    continue

                # 새로운 데이터 추가
                new_row = {"comments": new_comment, "hate": hate_type}
                existing_data = pd.concat([existing_data, pd.DataFrame([new_row])], ignore_index=True)
                generated_set.add(new_comment)
                additional_count -= 1

                # 중간 저장
                existing_data.to_csv(output_path, index=False, encoding="utf-8-sig")

        except Exception as e:
            print(f"오류 발생: {e}")
            continue

    return existing_data

# 각 파일 처리
for file_name in sorted(os.listdir(input_dir)):
    if file_name.endswith(".csv"):
        input_path = os.path.join(input_dir, file_name)
        output_file = f"ThreeShot_{file_name.split('_')[-1]}"
        output_path = os.path.join(output_dir, output_file)

        # 파일 로드
        gr_data = pd.read_csv(input_path)

        # 클래스별 100% 추가 생성
        current_counts = gr_data['hate'].value_counts()
        for hate_type, count in current_counts.items():
            additional_count = count  # 100% 증가
            gr_data = generate_data(hate_type, additional_count, gr_data, output_path)

        # 클래스 추가 후 확인
        final_counts = gr_data['hate'].value_counts()
        print(f"{output_path} - 모든 클래스의 데이터가 100% 증가했습니다. 최종 분포: {final_counts}")

# 최종 완료 메시지
print("모든 파일에 대한 데이터 생성 및 저장이 완료되었습니다.")


./Data/ThreeShot/ThreeShot_1.csv - 모든 클래스의 데이터가 100% 증가했습니다. 최종 분포: hate
none         88
offensive    62
hate         50
Name: count, dtype: int64
./Data/ThreeShot/ThreeShot_2.csv - 모든 클래스의 데이터가 100% 증가했습니다. 최종 분포: hate
none         94
offensive    58
hate         48
Name: count, dtype: int64
./Data/ThreeShot/ThreeShot_3.csv - 모든 클래스의 데이터가 100% 증가했습니다. 최종 분포: hate
none         84
hate         66
offensive    50
Name: count, dtype: int64
./Data/ThreeShot/ThreeShot_4.csv - 모든 클래스의 데이터가 100% 증가했습니다. 최종 분포: hate
none         100
offensive     60
hate          40
Name: count, dtype: int64
./Data/ThreeShot/ThreeShot_5.csv - 모든 클래스의 데이터가 100% 증가했습니다. 최종 분포: hate
none         80
offensive    78
hate         42
Name: count, dtype: int64
모든 파일에 대한 데이터 생성 및 저장이 완료되었습니다.


In [39]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import f1_score
import os

# 사용자 정의 데이터셋 클래스
class SimpleDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "label": torch.tensor(self.labels[idx], dtype=torch.long),
        }

# 경로 설정
base_train_dir = "./Data"
train_dirs = ["ZeroShot", "OneShot", "ThreeShot"]
test_file = "./Data/Test.csv"
summary_file = "./Data/Summary.csv"

# 모델 및 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # 3개 클래스
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 학습 및 평가 결과 저장
results = []

# 테스트 데이터 준비
test_data = pd.read_csv(test_file)
test_texts = test_data["comments"].tolist()
test_labels = test_data["hate"].map({"none": 0, "hate": 1, "offensive": 2}).tolist()
test_dataset = SimpleDataset(test_texts, test_labels, tokenizer, max_len=128)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 학습 루프
for train_dir_name in train_dirs:
    train_dir = os.path.join(base_train_dir, train_dir_name)
    for train_file in sorted(os.listdir(train_dir)):
        if train_file.endswith(".csv"):
            train_data = pd.read_csv(os.path.join(train_dir, train_file))
            train_texts = train_data["comments"].tolist()
            train_labels = train_data["hate"].map({"none": 0, "hate": 1, "offensive": 2}).tolist()

            train_dataset = SimpleDataset(train_texts, train_labels, tokenizer, max_len=128)
            train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

            # 옵티마이저 설정
            optimizer = AdamW(model.parameters(), lr=5e-5)

            # 학습
            model.train()
            for epoch in range(3):  # 3 에포크
                total_loss = 0
                for batch in train_loader:
                    input_ids = batch["input_ids"].to(device)
                    attention_mask = batch["attention_mask"].to(device)
                    labels = batch["label"].to(device)

                    optimizer.zero_grad()
                    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    loss.backward()
                    optimizer.step()

                    total_loss += loss.item()

                print(f"{train_dir_name}/{train_file} - Epoch {epoch + 1} - Loss: {total_loss:.4f}")

            # 평가
            model.eval()
            all_labels = []
            all_preds = []
            with torch.no_grad():
                for batch in test_loader:
                    input_ids = batch["input_ids"].to(device)
                    attention_mask = batch["attention_mask"].to(device)
                    labels = batch["label"].to(device)

                    outputs = model(input_ids, attention_mask=attention_mask)
                    preds = torch.argmax(outputs.logits, axis=1).cpu().numpy()
                    all_preds.extend(preds)
                    all_labels.extend(labels.cpu().numpy())

            f1_micro = f1_score(all_labels, all_preds, average="micro")
            f1_macro = f1_score(all_labels, all_preds, average="macro")
            print(f"Test Results for {train_dir_name}/{train_file}: F1 Micro = {f1_micro:.4f}, F1 Macro = {f1_macro:.4f}")
            results.append({"Dir": train_dir_name, "File": train_file, "F1 Micro": f1_micro, "F1 Macro": f1_macro})

# 결과 저장 또는 추가
if os.path.exists(summary_file):
    # 기존 파일이 있는 경우: 기존 데이터를 읽어서 새로운 데이터를 추가
    existing_results = pd.read_csv(summary_file)
    results_df = pd.concat([existing_results, pd.DataFrame(results)], ignore_index=True)
else:
    # 기존 파일이 없는 경우: 새로운 데이터만 저장
    results_df = pd.DataFrame(results)

results_df.to_csv(summary_file, index=False)
print(f"Summary updated and saved to {summary_file}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ZeroShot/ZeroShot_1.csv - Epoch 1 - Loss: 7.8127
ZeroShot/ZeroShot_1.csv - Epoch 2 - Loss: 7.4376
ZeroShot/ZeroShot_1.csv - Epoch 3 - Loss: 7.6101
Test Results for ZeroShot/ZeroShot_1.csv: F1 Micro = 0.3461, F1 Macro = 0.1917


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ZeroShot/ZeroShot_2.csv - Epoch 1 - Loss: 7.3556
ZeroShot/ZeroShot_2.csv - Epoch 2 - Loss: 7.6909
ZeroShot/ZeroShot_2.csv - Epoch 3 - Loss: 7.3627
Test Results for ZeroShot/ZeroShot_2.csv: F1 Micro = 0.3397, F1 Macro = 0.1690


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

ThreeShot/ThreeShot_3.csv - Epoch 1 - Loss: 7.3951
ThreeShot/ThreeShot_3.csv - Epoch 2 - Loss: 6.7365
ThreeShot/ThreeShot_3.csv - Epoch 3 - Loss: 6.1366
Test Results for ThreeShot/ThreeShot_3.csv: F1 Micro = 0.3524, F1 Macro = 0.2927


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ThreeShot/ThreeShot_4.csv - Epoch 1 - Loss: 7.3938
ThreeShot/ThreeShot_4.csv - Epoch 2 - Loss: 6.7265
ThreeShot/ThreeShot_4.csv - Epoch 3 - Loss: 6.3817
Test Results for ThreeShot/ThreeShot_4.csv: F1 Micro = 0.3546, F1 Macro = 0.2814


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ThreeShot/ThreeShot_5.csv - Epoch 1 - Loss: 6.9652
ThreeShot/ThreeShot_5.csv - Epoch 2 - Loss: 6.5791
ThreeShot/ThreeShot_5.csv - Epoch 3 - Loss: 6.3519
Test Results for ThreeShot/ThreeShot_5.csv: F1 Micro = 0.3694, F1 Macro = 0.3263
Summary updated and saved to ./Data/Summary.csv
